*Pipeline* que é como esperamos que faremos o estudo <br>
João Pedro Meirelles <br>
Pedro Daher <br>
Pedro Vero Fontes <br>
Jose Helio <br>
<b>Objetivo da entrega, chegar no teste de hipotese, explicar tudo, resumindo, um codigo bem explicado</b>

Comeco de tudo, precisamos de uma hipotese bem feita.
<h1>Hipotese</h1><br>
<h3> Interacao 1</h3><br>
Personagem com poderes mais raros, tendem a ser mais populares (aparecer mais em comics).
<h3> Interacao 2</h3><br>
Poderes mais raros geram um senso maior de importancia, pois são coisas que só ele consegue fazer, assim em certas situacoes onde tal poder eh necessario, menos super-herois poder ser inseridos pois o poder necessario eh mais raro.
<h3> Interacao 3</h3><br>
Como comics tem sempre um problema, ou vilao a ser derrotado, gera sempre necessidade de inumeros poderes serem utilizados, assim, quanto menos personagens terem os mesmos poderes ou semelhantes, mais vezes o personagem que possui um poder raro precisara ser chamado.

<h2>Preparacao</h2><br>
Pra que consigamos fazer nossa rede, precisamos primeiro montar uma tabla com os dados quantitativos, então os dados de cross over, pegando a tabela com os comics e os personagens dentro dela.<br>
Em conjunto com isso precisamos fazer tambem uma tabela apenas com os poderes e contabilizando quais poderes aparecem mais que outros, criando o que eu decrevi como "Unicos", que em jargões, seria degree talvez.<br>
Com as duas tabelas criadas acima, podemos então montar uma rede entre os personagens que aparecem em comics iguais, e tambem montar uma rede de personagens que aparecem em comics iguais e poderes.<br>
<h2>Estudo das redes</h2><br>
Na preparacao a gente "montou" duas tabelas principais pra montar a rede de nosso interesse, e com essa rede esperamos identificar algumas coisas.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import networkx as nx
import freeman as fm

In [9]:
from csv import reader
from unidecode import unidecode

personagens = {}
edges = []
comics = {}
poderes = {}

with open("characters.csv",encoding="utf-8") as file:
    file.readline()
    for i, row in enumerate(reader(file), 2):
        charId = row[0]
        nome = row[1]
        personagens[int(charId)] = nome

with open("comics.csv",encoding="utf-8") as file:
    file.readline()
    for i, row in enumerate(reader(file), 2):
        comicId = row[0]
        title = row[1]
        comics[int(comicId)] = title

with open("charactersToComics.csv",encoding="utf-8") as file:
    file.readline()
    for i, row in enumerate(reader(file), 2):
        edges.append([row[0],row[1]])

In [10]:
with open('charComic.gml','w') as file:
    file.write('graph [\n')
    file.write('    directed 1\n')
    
    for char in personagens.keys(): # Cria os nodes dos personagens
        if '"' in personagens[char]:
            personagens[char] = personagens[char].replace('"'," ")
        file.write('    node [\n')
        file.write(f'        id {char}\n')
        file.write(f'        name "{unidecode(personagens[char])}"\n')
        file.write('        tipo "character"\n')
        file.write('    ]\n')
        
    for comic in comics.keys(): # Cria os nodes dos comics
        if '"' in comics[comic]:
            comics[comic] = comics[comic].replace('"'," ")
        file.write('    node [\n')
        file.write(f'        id {comic}\n')
        file.write(f'        name "{unidecode(comics[comic])}"\n')
        file.write('        tipo "comic"\n')
        file.write('    ]\n')
    
    for (s, t) in edges:        # Monta as ligacoes entre nodes
        file.write('    edge [\n')
        file.write(f'        source {s}\n')
        file.write(f'        target {t}\n')
        file.write('        weight 1\n')
        file.write('    ]\n')
        
    file.write(']\n') # Fecha o graph

In [11]:
def load():
    g = fm.load('charComic.gml')

    # Remover todas as arestas com peso menor ou igual a 0.5.
    # Precisamos de dois loops, pois não é uma boa ideia
    # tirar algo de um conjunto enquanto iteramos nele.
#     removed = []
#     for n, m in g.edges:
#         if g.edges[n, m]['weight'] <= 0.5:
#             removed.append((n, m))
#     for n, m in removed:
#         g.remove_edge(n, m)

    # Remover todos os nós que ficaram isolados depois da
    # remoção das arestas, para melhorar a visualização.
    removed = []
    for n in g.nodes:
        if not g.degree(n):
            removed.append(n)
    for n in removed:
        g.remove_node(n)

    return g

def draw(g):
    # Deixar os nós menores e fazer os nomes deles
    # aparecer apenas quando o mouse passa por cima.
    g.set_all_nodes(size=10, labpos='hover')

    # Deixar as arestas semi-transparentes para
    # a visualização ficar um pouco mais leve.
    g.set_all_edges(color=(0, 0, 0, 0.25))

    # Usar um algoritmo especial para posicionar
    # os nós de acordo com grupos.
    g.move('kamada_kawai')

    g.draw()

In [12]:
G = load()

NetworkXError: expected an int, float, string or '[', found 'BATTLE' at (8420, 52)

In [ ]:
g.draw()

<h2>Formulas uteis (Aula 4)</h2>

In [ ]:
from math import log

def proportion(g, n, m):
    return 1 / g.degree(n)

def local_effsize(g, n, m):
    s = 1
    for k in g.neighbors(n):
        if g.has_edge(k, m):
            s -= proportion(g, n, k)
    return s

def effsize(g, n):
    s = 0
    for m in g.neighbors(n):
        s += local_effsize(g, n, m)
    return s

def local_constraint(g, n, m):
    s = proportion(g, n, m)
    for k in g.neighbors(n):
        if g.has_edge(k, m):
            s += proportion(g, n, k) * proportion(g, k, m)
    return s**2

def constraint(g, n):
    if g.degree(n) == 0:
        return 2
    s = 0
    for m in g.neighbors(n):
        s += local_constraint(g, n, m)
    return s

def hierarchy(g, n):
    c = constraint(g, n)
    N = g.number_of_nodes()
    s = 0
    for m in g.neighbors(n):
        f = local_constraint(g, n, m) / (c / N)
        s += f * log(f)
    return s / (N * log(N))

In [ ]:
print('calculando centralidade degree...')
dc = nx.degree_centrality(g)

print('calculando centralidade betweenness...')
bc = nx.betweenness_centrality(g)

print('calculando centralidade closeness...')
cc = nx.closeness_centrality(g)

print('calculando centralidade eigenvector...')
ec = nx.eigenvector_centrality(g)

Calculando metricas novas

In [ ]:
print('calculando efficient size...')
eff = {}
for n in g.nodes:
    eff[n] = effsize(g, n)

print('calculando constraint...')
con = {}
for n in g.nodes:
    con[n] = constraint(g, n)

print('calculando hierarchy...')
hie = {}
for n in g.nodes:
    hie[n] = hierarchy(g, n)

Armazena dados num DataFrame

In [ ]:
data = pd.DataFrame({
    'nodes': [n for n in g.nodes],
    'substantivo': [g.nodes[n]['substantivo'] for n in g.nodes],
    'degree centrality': [dc[n] for n in g.nodes],
    'betweenness centrality': [bc[n] for n in g.nodes],
    'closeness centrality': [cc[n] for n in g.nodes],
    'eigenvector centrality': [ec[n] for n in g.nodes],
    'efficient size': [eff[n] for n in g.nodes],
    'constraint': [con[n] for n in g.nodes],
    'hierarchy': [hie[n] for n in g.nodes],
})

fm.resize_all_plots(800, 1000)

In [ ]:
sns.scatterplot(data=data, x='constraint', y='betweenness centrality');

In [ ]:
sns.scatterplot(data=data, x='degree centrality', y='efficient size');

In [ ]:
sns.boxplot(data=data, x='betweenness centrality', y='name');